<a href="https://colab.research.google.com/github/AkseliManninen/A-Star-Search-Algorithm-Visualized/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing and importing libraries

In [1]:
# Importing libraries
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.utils.data as data

from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Configuration

In [2]:
# Select the device for training (use GPU if you have one)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
  print("Using a GPU")
else:
  print("Using a CPU")

Using a GPU


In [3]:
# Changing the working directory
from google.colab import drive
drive.mount('/content/drive')

# Path to the "Tutkimusprojekti" folder in the root of Google Drive
google_drive_path = '/content/drive/My Drive/Opinnot/Diplomityö/Koodi/dippa/models'

# Change the current directory to the "Tutkimusprojekti" folder
os.chdir(google_drive_path)

# Check the current working directory to verify the change
data_dir = os.getcwd()
print("Current working directory:", data_dir)

Mounted at /content/drive
Current working directory: /content/drive/My Drive/Opinnot/Diplomityö/Koodi/dippa/models


In [198]:
# Define the grid size to be either 1000 or 250
grid_size = 1000

df = pd.read_csv(f'/content/dataset_{grid_size}.csv')

df = df[["grid_1000", "Month_Year", "RAIN", "TEMP", "Q1", "Q2", "Q3", "Q4", "PAVED", "DIRT", "COUNT"]]

# Define the input size based on the number of features
input_size = df.shape[1] - 2
df.head()

,grid_1000,Month_Year,RAIN,TEMP,Q1,Q2,Q3,Q4,PAVED,DIRT,COUNT
0,0.0,2014-01-01,49.9,-6.4,1,0,0,0,0,0,0.0
1,0.0,2014-02-01,29.4,-0.1,1,0,0,0,0,0,0.0
2,0.0,2014-03-01,32.2,2.0,1,0,0,0,0,0,0.0
3,0.0,2014-04-01,9.9,6.0,0,1,0,0,0,0,0.0
4,0.0,2014-05-01,59.1,10.8,0,1,0,0,0,0,0.0


In [199]:
# train-test split for time series
train = df[df["Month_Year"].str.contains("2022") == False]
train.reset_index(drop=True, inplace=True)

test = df[df["Month_Year"].str.contains("2022")]
test.reset_index(drop=True, inplace=True)

test.head()

,grid_1000,Month_Year,RAIN,TEMP,Q1,Q2,Q3,Q4,PAVED,DIRT,COUNT
0,0.0,2022-01-01,102.3,-2.6,1,0,0,0,0,0,0.0
1,0.0,2022-02-01,127.5,-1.4,1,0,0,0,0,0,0.0
2,0.0,2022-03-01,14.9,0.8,1,0,0,0,0,0,0.0
3,0.0,2022-04-01,34.7,4.1,0,1,0,0,0,0,0.0
4,0.0,2022-05-01,50.7,9.8,0,1,0,0,0,0,0.0


In [200]:
# Normalize dataset
normalized_train = train.copy()
normalized_train.iloc[:, 2:] = (normalized_train.iloc[:, 2:]-normalized_train.iloc[:, 2:].mean())/normalized_train.iloc[:, 2:].std()

normalized_test = test.copy()
normalized_test.iloc[:, 2:] = (normalized_test.iloc[:, 2:]-normalized_test.iloc[:, 2:].mean())/normalized_test.iloc[:, 2:].std()
normalized_test.head()

,grid_1000,Month_Year,RAIN,TEMP,Q1,Q2,Q3,Q4,PAVED,DIRT,COUNT
0,0.0,2022-01-01,1.372602,-1.251601,1.731843,-0.577281,-0.577281,-0.577281,-0.933714,-1.10915,-0.342998
1,0.0,2022-02-01,2.199114,-1.096897,1.731843,-0.577281,-0.577281,-0.577281,-0.933714,-1.10915,-0.342998
2,0.0,2022-03-01,-1.493955,-0.813272,1.731843,-0.577281,-0.577281,-0.577281,-0.933714,-1.10915,-0.342998
3,0.0,2022-04-01,-0.844552,-0.387835,-0.577281,1.731843,-0.577281,-0.577281,-0.933714,-1.10915,-0.342998
4,0.0,2022-05-01,-0.319782,0.347010,-0.577281,1.731843,-0.577281,-0.577281,-0.933714,-1.10915,-0.342998


In [202]:
def create_dataset(original_dataset, normalized_dataset, lookback):
    """Transform a time series into a prediction dataset

    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
      """

    X_batches, y_batches = [], []
    batch_size = 64  # Adjust batch size according to your available memory
    for i in range(0, len(original_dataset) - lookback, batch_size):
        batch_X, batch_y = [], []
        for j in range(i, min(i + batch_size, len(original_dataset) - lookback)):
            if original_dataset[f"grid_{grid_size}"].iloc[j] == original_dataset[f"grid_{grid_size}"].iloc[j + lookback]:
                feature = normalized_dataset.iloc[:, 2:].iloc[j:j + lookback].to_numpy() # Features, including COUNT
                target = original_dataset["COUNT"].iloc[j + lookback] # Target is COUNT for the next month
                batch_X.append(feature)
                batch_y.append(target)
        X_batches.extend(torch.tensor(batch_X).float())
        y_batches.extend(torch.tensor(batch_y).float())
    return X_batches, y_batches

lookback = 6
X_train_batches, y_train_batches = create_dataset(train, normalized_train, lookback=lookback)
X_test_batches, y_test_batches = create_dataset(test, normalized_test, lookback=lookback)


In [203]:
X_train = torch.stack(X_train_batches)
X_train = X_train.to(device)
y_train = torch.stack(y_train_batches)
y_train = y_train.to(device)
X_test = torch.stack(X_test_batches)
y_test = torch.stack(y_test_batches)

In [204]:
print(X_train.shape) # Samples, Months, Features

torch.Size([31230, 6, 9])


# LSTM Simple

In [37]:
class AirModel(nn.Module):
    def __init__(self):
        super().__init__()
        print(input_size)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=128, num_layers=4, batch_first=True)
        self.linear = nn.Linear(128, 1)

    def forward(self, x):
        """
        x shape: batch_size x sequence_length (look_back) x num_features
        """
        x, _ = self.lstm(x)
        x = self.linear(x) # batch_size x sequence_length x 1
        x = x[:, -1, :]
        x = x.T.flatten()

        return x

In [ ]:
# Define custom weights for each target value
def get_weights(y_batch):
    weights = torch.zeros_like(y_batch, dtype=torch.float)
    weights[y_batch == 0] = 0.1
    weights[y_batch == 1] = 0.2
    weights[y_batch == 2] = 0.25
    weights[y_batch >= 3] = 0.5
    return weights

model = AirModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.MSELoss(reduction='none')  # Using MSE loss with 'none' reduction
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=30)

n_epochs = 3
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)

        # Calculate custom weights for each sample in the batch
        weights = get_weights(y_batch)

        optimizer.zero_grad()
        weighted_loss = torch.mean(weights * loss)  # Apply weights to the loss
        weighted_loss.backward()
        optimizer.step()

    # Validation
    if epoch % 1 != 0:
        continue
    model.eval()
    with torch.no_grad():
        print("weighted loss", weighted_loss)
        y_pred = model(X_train)
        train_loss = loss_fn(y_pred, y_train)
        train_rmse = np.sqrt(torch.mean(train_loss))
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        test_rmse = np.sqrt(torch.mean(test_loss))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch+1, train_rmse, test_rmse))


15
weighted loss tensor(0.2442, device='cuda:0', grad_fn=<MeanBackward0>)


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
print(len(y_pred))
print("Pred round:\n",y_pred.T.round()[500:550], "\n")

#print(y_pred[550:600], "\n")

print("Test:\n",y_test[500:550], "\n")

print("correct", torch.sum(y_pred.round() == y_test.round()))
print("all", len(y_pred))
print("zeros", torch.sum(y_test.round() == 0))

print(torch.sum((y_pred.round() == 1) & (y_pred.round() == y_test.round())))
print(torch.sum((y_pred.round() == 1)))

print(torch.sum((y_test.round() == 1)))

In [ ]:
print(torch.sum((y_pred.round() >= 1) & (y_pred.round() == y_test.round())))

tensor(116)


In [ ]:
print(len(y_test))
print((y_test == 0).sum())
print((y_test == 1).sum())
print((y_test == 2).sum())
print((y_test == 3).sum())
print((y_test == 4).sum())
print((y_test == 5).sum())
print((y_test == 6).sum())
print((y_test == 7).sum())

1388
tensor(1202)
tensor(116)
tensor(45)
tensor(20)
tensor(5)
tensor(0)
tensor(0)
tensor(0)


In [ ]:
print(y_pred.round().unique())
print(y_pred.unique())

tensor([-0., 1., 2., 3.])
tensor([-4.3150e-02, -3.8366e-02, -3.7188e-02, -3.6721e-02, -1.9346e-02,
         1.6920e-03,  3.0016e-02,  4.0333e-02,  4.3748e-02,  5.8640e-02,
         6.8418e-02,  7.6406e-02,  8.1269e-02,  8.4553e-02,  9.0865e-02,
         9.4020e-02,  9.8343e-02,  1.0699e-01,  1.0708e-01,  1.0810e-01,
         1.1209e-01,  1.1800e-01,  1.1931e-01,  1.1936e-01,  1.2760e-01,
         1.2890e-01,  1.3148e-01,  1.4025e-01,  1.4196e-01,  1.4227e-01,
         1.4475e-01,  1.4796e-01,  1.4803e-01,  1.5079e-01,  1.6175e-01,
         1.6288e-01,  1.6591e-01,  1.7043e-01,  1.7267e-01,  1.8344e-01,
         1.9228e-01,  1.9714e-01,  1.9731e-01,  2.0126e-01,  2.0397e-01,
         2.0582e-01,  2.1532e-01,  2.1694e-01,  2.1815e-01,  2.2199e-01,
         2.2321e-01,  2.3637e-01,  2.3675e-01,  2.4794e-01,  2.4994e-01,
         2.5192e-01,  2.6014e-01,  2.6206e-01,  2.6726e-01,  2.6776e-01,
         2.7067e-01,  2.7726e-01,  2.9201e-01,  2.9223e-01,  2.9294e-01,
         2.9362e-01,  2.9

# LSTM simple 2

In [205]:
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        print(input_size)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=100, num_layers=5, batch_first=True)
        self.linear = nn.Linear(100, 100)
        self.linear2 = nn.Linear(100, 100)
        self.linear3 = nn.Linear(100, 100)
        self.linear4 = nn.Linear(100, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        """
        x shape: batch_size x sequence_length (look_back) x num_features
        """
        x, _ = self.lstm(x)

        x = self.linear(x)
        x = self.linear2(x)

        x = self.dropout(x)
        x = self.relu(x)

        x = self.linear3(x)
        x = self.linear4(x)

        x = x[:, -1, :]
        x = x.T.flatten()

        #x = x.round()

        return x

In [206]:
# Define custom weights for each target value
def get_weights(y_batch):
    weights = torch.zeros_like(y_batch, dtype=torch.float)
    weights[y_batch == 0] = 1 #0.2 #0.2
    weights[y_batch == 1] = 1 #0.3 #0.25
    weights[y_batch == 2] = 1 #0.4 #0.25
    weights[y_batch >= 3] = 1 #0.5 #0.3
    return weights

model = LSTM().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.MSELoss(reduction='none')  # Using MSE loss with 'none' reduction
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=30)

n_epochs = 20
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)

        # Calculate custom weights for each sample in the batch
        weights = get_weights(y_batch)

        optimizer.zero_grad()
        weighted_loss = torch.mean(weights * loss)  # Apply weights to the loss
        weighted_loss.backward()
        optimizer.step()

    # Validation
    if epoch % 1 != 0:
        continue
    model.eval()
    with torch.no_grad():
        print("weighted loss", weighted_loss)
        y_pred = model(X_train)
        y_pred, y_test, X_test = y_pred.to(device), y_test.to(device), X_test.to(device)
        train_loss = loss_fn(y_pred, y_train)
        train_rmse = torch.sqrt(torch.mean(train_loss))
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        test_rmse = torch.sqrt(torch.mean(test_loss))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch+1, train_rmse, test_rmse))


9
weighted loss tensor(1.1314, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 1: train RMSE 0.8097, test RMSE 0.7866
weighted loss tensor(0.5663, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 2: train RMSE 0.7965, test RMSE 0.7605
weighted loss tensor(0.8213, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 3: train RMSE 0.8071, test RMSE 0.8681
weighted loss tensor(0.0647, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 4: train RMSE 0.7945, test RMSE 0.6923
weighted loss tensor(0.2106, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 5: train RMSE 0.8100, test RMSE 0.6439
weighted loss tensor(1.1498, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 6: train RMSE 0.7889, test RMSE 0.6885
weighted loss tensor(0.3978, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 7: train RMSE 0.7887, test RMSE 0.6934
weighted loss tensor(0.8800, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 8: train RMSE 0.7874, test RMSE 0.7909
weighted loss tensor(0.6909, device='cuda:0', grad_fn=<MeanBackward0>)

In [207]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
print(len(y_pred))
print("Pred round:\n",y_pred.T.round()[500:550], "\n")

print("Test:\n",y_test[500:550], "\n")

print("Correct", torch.sum(y_pred.round() == y_test.round()))
print("all", len(y_pred))
print("zeros", torch.sum(y_test.round() == 0))

print(torch.sum((y_pred.round() == 1) & (y_pred.round() == y_test.round())))
print(torch.sum((y_pred.round() == 0)))

print(torch.sum((y_test.round() == 1)))

2082
Pred round:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.],
       device='cuda:0') 

Test:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 2., 0., 0., 1., 0., 1., 0., 0.],
       device='cuda:0') 

Correct tensor(1596, device='cuda:0')
all 2082
zeros tensor(1797, device='cuda:0')
tensor(59, device='cuda:0')
tensor(1569, device='cuda:0')
tensor(190, device='cuda:0')


In [208]:
print(torch.sum((y_pred.round() == 0) & (y_pred.round() == y_test.round())))
print(torch.sum((y_pred.round() >= 1) & (y_pred.round() == y_test.round())))

tensor(1513, device='cuda:0')
tensor(83, device='cuda:0')


# Old TARPML

In [20]:
class TrafficAccidentLSTM(nn.Module):
    def __init__(self, input_size_lstm, hidden_size_lstm, num_layers_lstm, input_size_fc, hidden_size_fc, num_layers_fc, output_size, dropout_rate):
        super(TrafficAccidentLSTM, self).__init__()

        self.lstm = nn.LSTM(input_size=input_size_lstm, hidden_size=hidden_size_lstm, num_layers=num_layers_lstm, batch_first=True)

        self.fc_layers = nn.ModuleList()
        self.fc_layers.append(nn.Linear(7, hidden_size_fc))
        for _ in range(num_layers_fc - 1):
            self.fc_layers.append(nn.Linear(hidden_size_fc, hidden_size_fc))

        self.output_layer = nn.Linear(hidden_size_fc, output_size)

        self.dropout = nn.Dropout(dropout_rate)

        self.relu = nn.ReLU()

    def forward(self, input_seq_lstm, input_fc):

        lstm_out, _ = self.lstm(input_seq_lstm)

        lstm_out = lstm_out[:, -1, :]  # Taking the output of the last timestep

        input_fc = input_fc[:, -1, :]

        combined_input = torch.cat((lstm_out, input_fc), dim=1)

        for fc_layer in self.fc_layers:
            combined_input = self.dropout(combined_input)
            combined_input = fc_layer(combined_input)

        output = self.output_layer(combined_input)

        return output

In [21]:
# Define custom weights for each target value
def get_weights(y_batch):
    weights = torch.zeros_like(y_batch, dtype=torch.float)
    weights[y_batch == 0] = 0.1
    weights[y_batch == 1] = 0.2
    weights[y_batch == 2] = 0.25
    weights[y_batch >= 3] = 0.5
    return weights

input_size_lstm = 10 # num features
hidden_size_lstm = 2
num_layers_lstm = 4

input_size_fc = 15-4
hidden_size_fc = 5
num_layers_fc = 3

output_size = 1  # Predicted traffic accident risk (frequency)

dropout_rate = 0.5

model = TrafficAccidentLSTM(input_size_lstm, hidden_size_lstm, num_layers_lstm, input_size_fc, hidden_size_fc, num_layers_fc, output_size, dropout_rate).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.MSELoss(reduction='none')  # Using MSE loss with 'none' reduction
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=30)

n_epochs = 3
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred = model(X_batch[:,:,:10], X_batch[:,:,-5:])
        loss = loss_fn(y_pred, y_batch)

        # Calculate custom weights for each sample in the batch
        weights = get_weights(y_batch)

        optimizer.zero_grad()
        weighted_loss = torch.mean(weights * loss)  # Apply weights to the loss
        weighted_loss.backward()
        optimizer.step()

    # Validation
    if epoch % 1 != 0:
        continue
    model.eval()
    with torch.no_grad():
        print("weighted loss", weighted_loss)
        y_pred = model(X_train[:,:,:10], X_train[:,:,-5:])
        train_loss = loss_fn(y_pred, y_train)
        train_rmse = torch.sqrt(torch.mean(train_loss))
        #test_loss = loss_fn(y_pred, y_test)
        #test_rmse = np.sqrt(torch.mean(test_loss))
        test_rmse = -1
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch+1, train_rmse, test_rmse))


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([30])) that is different to the input size (torch.Size([30, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([26])) that is different to the input size (torch.Size([26, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([30536])) that is different to the input size (torch.Size([30536, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(

weighted loss tensor(5.0166, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 1: train RMSE 1.3806, test RMSE -1.0000
weighted loss tensor(0.8648, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 2: train RMSE 1.3176, test RMSE -1.0000
weighted loss tensor(0.1350, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 3: train RMSE 1.3055, test RMSE -1.0000


In [22]:
model.eval()
with torch.no_grad():
    X_test = X_test.to(device)
    y_pred = model(X_test[:,:,:10], X_test[:,:,-5:])

print(y_pred.shape)

y_pred=y_pred.to(device)
y_test=y_test.to(device)

print(y_test.round())
print(y_pred.T.round())

#comparison_tensor = torch.round(y_pred).eq(torch.round(y_test)).int()
#comparison_list = comparison_tensor.tolist()
#print(comparison_list)

y_pred = y_pred.flatten()
print("ypred:",y_pred.size())
print("ytest:",y_test.size())


l = torch.round(y_pred).eq(torch.round(y_test)).int()

print(l)

# Get the number of 0s and 1s in l
num_zeros = torch.sum(l == 0).item()
num_ones = torch.sum(l == 1).item()

print("Number of 0s:", num_zeros)
print("Number of 1s:", num_ones)

print(l.size())

#print((y_pred.round() == y_test.round()))
#print("all", len(y_pred))
#print("zeros", torch.sum(y_test.round() == 0))

#print(torch.sum((y_pred.round() == 1) & (y_pred.round() == y_test.round())))
#print(torch.sum((y_pred.round() == 1)))

#print(torch.sum((y_test.round() == 1)))
print(torch.sum((y_pred.round() == 1)))
print(torch.sum((y_test.round() == 0)))

torch.Size([1388, 1])
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
ypred: torch.Size([1388])
ytest: torch.Size([1388])
tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0', dtype=torch.int32)
Number of 0s: 225
Number of 1s: 1163
torch.Size([1388])
tensor(64, device='cuda:0')
tensor(1202, device='cuda:0')


# TARPML

In [170]:
class TARPML(nn.Module):
    def __init__(self, input_size_lstm, hidden_size_lstm, num_layers_lstm, input_size_fc, hidden_size_fc, num_layers_fc, output_size, dropout_rate):
        super(TARPML, self).__init__()

        self.lstm = nn.LSTM(input_size=input_size_lstm, hidden_size=hidden_size_lstm, num_layers=num_layers_lstm, batch_first=True)

        self.fc_layers = nn.ModuleList()

        self.fc_layers.append(nn.Linear(hidden_size_lstm + input_size_fc, hidden_size_fc))
        for _ in range(num_layers_fc - 1):
            self.fc_layers.append(nn.Linear(hidden_size_fc, hidden_size_fc))

        #self.output_layer = nn.Linear(hidden_size_fc, output_size)
        self.output_layer = nn.Linear(hidden_size_fc, output_size)


        self.dropout = nn.Dropout(dropout_rate)

        self.relu = nn.ReLU()

    def forward(self, input_seq_lstm, input_fc):

        lstm_out, _ = self.lstm(input_seq_lstm)
        #print("4")
        combined_input = torch.cat((lstm_out, input_fc), dim=2)
        #print(combined_input.shape)
        #print("5")

        #print(combined_input.shape)
        for fc_layer in self.fc_layers:
          combined_input = self.dropout(combined_input)
          #print("6")
          combined_input = fc_layer(self.relu(combined_input))
          #print("7")

        output = self.output_layer(combined_input)
        #print("8")

        output = output[:, -1, :]

        return output

In [171]:
# Define custom weights for each target value
def get_weights(y_batch):
    weights = torch.zeros_like(y_batch, dtype=torch.float)
    weights[y_batch == 0] = 0.2
    weights[y_batch == 1] = 0.25
    weights[y_batch == 2] = 0.25
    weights[y_batch >= 3] = 0.3
    return weights

input_size_lstm = 7 # num time-variant features
hidden_size_lstm = 100 # num neurons
num_layers_lstm = 4 # num lstm neurons

input_size_fc = 2 # num time-invariant features 8
hidden_size_fc = 100 # num neurons
num_layers_fc = 3 # num fully connected layers

output_size = 1  # Predicted traffic accident risk (frequency)

dropout_rate = 0.2

model = TARPML(input_size_lstm, hidden_size_lstm, num_layers_lstm, input_size_fc, hidden_size_fc, num_layers_fc, output_size, dropout_rate).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.00001)
loss_fn = nn.MSELoss(reduction='none')  # Using MSE loss with 'none' reduction
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=30)

n_epochs = 20
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        lstm_input = X_batch[:, :, [0, 1, 2, 3, 4, 5, 8]]
        fc_input = X_batch[:,:, [6, 7]]

        y_pred = model(lstm_input, fc_input)
        loss = loss_fn(y_pred, y_batch)

        # Calculate custom weights for each sample in the batch
        weights = get_weights(y_batch)

        optimizer.zero_grad()
        weighted_loss = torch.mean(weights * loss)  # Apply weights to the loss
        weighted_loss.backward()
        optimizer.step()

    # Validation
    if epoch % 1 != 0:
        continue
    model.eval()
    with torch.no_grad():
        print("weighted loss", weighted_loss)
        y_pred = model(X_train[:, :, [0,1, 2, 3, 4, 5, 8]], X_train[:,:, [6, 7]]) #6, 7
        train_loss = loss_fn(y_pred, y_train)
        train_rmse = torch.sqrt(torch.mean(train_loss))
        #test_loss = loss_fn(y_pred, y_test)
        #test_rmse = np.sqrt(torch.mean(test_loss))
        test_rmse = -1
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch+1, train_rmse, test_rmse))


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([30])) that is different to the input size (torch.Size([30, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([26])) that is different to the input size (torch.Size([26, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([30536])) that is different to the input size (torch.Size([30536, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(

weighted loss tensor(0.2466, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 1: train RMSE 1.2821, test RMSE -1.0000
weighted loss tensor(1.2417, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 2: train RMSE 1.2858, test RMSE -1.0000
weighted loss tensor(0.7458, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 3: train RMSE 1.2805, test RMSE -1.0000
weighted loss tensor(2.1858, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 4: train RMSE 1.2837, test RMSE -1.0000
weighted loss tensor(1.5259, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 5: train RMSE 1.2793, test RMSE -1.0000
weighted loss tensor(0.4804, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 6: train RMSE 1.2822, test RMSE -1.0000
weighted loss tensor(0.2801, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 7: train RMSE 1.2796, test RMSE -1.0000
weighted loss tensor(0.1359, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch 8: train RMSE 1.2785, test RMSE -1.0000
weighted loss tensor(0.2360, device='cuda:0', grad_fn=<MeanBackw

In [169]:
model.eval()
with torch.no_grad():
    X_test = X_test.to(device)
    y_pred = model(X_test[:, :, [0,1, 2, 3, 4, 5, 8]], X_test[:,:,[6, 7]])

print(y_pred.shape)

y_pred=y_pred.to(device)
y_test=y_test.to(device)

print(y_test.round())
print(y_pred.T.round())

#comparison_tensor = torch.round(y_pred).eq(torch.round(y_test)).int()
#comparison_list = comparison_tensor.tolist()
#print(comparison_list)

y_pred = y_pred.flatten()
print("ypred:",y_pred.size())
print("ytest:",y_test.size())


l = torch.round(y_pred).eq(torch.round(y_test)).int()

print(l)

# Get the number of 0s and 1s in l
num_zeros = torch.sum(l == 0).item()
num_ones = torch.sum(l == 1).item()

print("Number of 0s:", num_zeros)
print("Number of 1s:", num_ones)

print(l.size())

#print((y_pred.round() == y_test.round()))
#print("all", len(y_pred))
#print("zeros", torch.sum(y_test.round() == 0))

#print(torch.sum((y_pred.round() == 1) & (y_pred.round() == y_test.round())))
#print(torch.sum((y_pred.round() == 1)))

print(torch.sum((y_pred.round() == 0)))
print(torch.sum((y_pred.round() == 1)))
print(torch.sum((y_pred.round() == 2)))

torch.Size([1388, 1])
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
tensor([[1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0')
ypred: torch.Size([1388])
ytest: torch.Size([1388])
tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0', dtype=torch.int32)
Number of 0s: 1272
Number of 1s: 116
torch.Size([1388])
tensor(0, device='cuda:0')
tensor(1388, device='cuda:0')
tensor(0, device='cuda:0')
